# Notes on Spatial Information for StochSS

When setting up a spatial problem in StochSS information about the geometry needs to be provided through two files: a
file containing the mesh information in DOLFIN XML format and a text file containing information about subdomains. The
file with subdomain information is optional in general (if it is not provided the entire mesh will belong to one subdomain).
Below is a tutorial on how to provide that information in a variety of cases.

## Mesh Library 

The first and simpilest scenario is using one of the default built­in meshes provided by StochSS in the Mesh Library.
These include simple example mesh and subdomain combinations that are common in biological modeling. For example
there is a unit sphere with a membrane (i.e. the volume is one subdomain and the surface is another), a unit cube witha
membrane, a cylinder with the two ends being different subdomains among others. If for a given problem these are not
sufficient then mesh and subdomain information can be provided by attaching files. One simple case where creating and
attaching a file may be necessary could involve using the DOLFIN built in meshes but with different subdomains than
provided in the Mesh Library. This can be done as follows.
## Using DOLFIN Built-In Geometry with Subdomains Outside of Mesh
Library
For example consider a sphere with three different subdomains: the top half of the membrane, the bottom half of the
membrane and the volume. This can be done using pyurdme as follows (pyurdme and FEniCS are software packages
that are automatically installed during the StochSS installation and DOLFIN is a library within FEniCS).
First the appropriate libraries need to be imported:

In [ ]:
FILENAME = "sphere_top_bot_mesh.xml"
SD_FILENAME = "sphere_top_botsubdomains.txt"

In [ ]:
import dolfin
import mshr
import os
import pyurdme

In [ ]:
class top_half_membrane(dolfin.SubDomain):
    def inside(self,x,on_boundary):
        return on_boundary and x[2]>=0
class bottom_half_membrane(dolfin.SubDomain):
    def inside(self,x,on_boundary):
        return on_boundary and x[2]<=0

In [ ]:
class sphere_top_bot(pyurdme.URDMEModel):
    def __init__(self,model_name="polarization"):
        pyurdme.URDMEModel.__init__(self,model_name)
        #DefineGeometry
        sphere=mshr.Sphere(dolfin.Point(0.0,0.0,0.0),1.0)
        self.mesh=pyurdme.URDMEMesh(mesh=mshr.generate_mesh(sphere,10))
        cell_function=dolfin.CellFunction("size_t",self.mesh)

        self.add_subdomain(top_half_membrane(),2)
        self.add_subdomain(bottom_half_membrane(),3)
        #Defineinitialpopulations
        self.set_initial_condition_scatter({},[1])

In [ ]:
model=sphere_top_bot()

In [ ]:
sd=model.get_subdomain_vector()

In [ ]:
#This is a dolfin xml file with mesh information
dolfin.File(FILENAME) << model.mesh

In [ ]:
import base64
import IPython.display
with open(FILENAME) as fd:
    b64 = base64.b64encode(fd.read())
    payload = b64.decode()
    html = '<H1><a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a></H1>'
    html = html.format(payload=payload,title="Click here to Download Mesh",filename=FILENAME)
IPython.display.HTML(html)

In [ ]:
#This is the file to capture information about subdomains
with open(SD_FILENAME,'w') as sfd:
    for ndx,val in enumerate(sd):
        sfd.write("{0},{1}\n".format(ndx,val))
import base64
import IPython.display
with open(SD_FILENAME) as fd:
    b64 = base64.b64encode(fd.read())
    payload = b64.decode()
    html = '<H1><a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a></H1>'
    html = html.format(payload=payload,title="Click here to Download SD file",filename=SD_FILENAME)
IPython.display.HTML(html)

In [ ]:
#  IF you want to visualize your mesh, execute the following two cells

In [ ]:
# Code to setup javascript display libraries
import os.path
import IPython.display
with open(os.path.join(os.path.dirname(pyurdme.__file__),'data/three.js_templates/js/three.js')) as fd:
    bufa = fd.read()
with open(os.path.join(os.path.dirname(pyurdme.__file__),'data/three.js_templates/js/render.js')) as fd:
    bufb = fd.read()
with open(os.path.join(os.path.dirname(pyurdme.__file__),'data/three.js_templates/js/OrbitControls.js')) as fd:
    bufc = fd.read()
IPython.display.display(IPython.display.HTML('<script>'+bufa+bufc+bufb+'</script>'))

In [ ]:
model.mesh